In [45]:
import findspark
from pyspark.sql import SparkSession

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [46]:
sc = spark.sparkContext

### Transformaciones en un RDD

Los RDDs son inmutables y cada operación crea un nuevo RDD. Las dos operaciones principales que se pueden realizar en una red son transformaciones y acciones:

   >-**Transformaciones:** cambian los elementos en el RDD. Algunos ejemplos pueden ser dividir el elemento de entrada, filtrar los elementos y realizar cálculos de algún tipo. Varias transformaciones se pueden realizar en una secuencia, sin embargo, no se lleva a cabo ninguna ejecución durante la planificación. Para las transformaciones spark las agrega al DAC de calculo. DAG proviene de Directed Acyclic Graph y sólo cuando el controlador solicita algunos datos de este DAG realmente se ejecuta; a esto se le llama evaluación perezosa o lazy evaluation.

   >-**Acciones:** son operaciones que activan los cálculos. Hasta que se encuentra una operación de acción el plan de ejecución dentro del programa Spark se crea en forma de DAG, por tanto, no se hará nada hasta que se realice una acción dentro del DAG. Podría haber varias transformaciones de todo tipo dentro del plan de ejecución, pero no sucede nada hasta que realicemos una acción. Cualquier tipo de variables que nosotros podamos tener en memoria acelerará grandemente nuestras aplicaciones de Spark, debido a que no se tendrá que ejecutar todo el DAG para reconstruir el RDD que querramos en ese momento.




#### Tipos de transformaciones

Las transformaciones se pueden dividir en cuatro categorías:

   >-**Transformaciones generales:** son funciones de transformación que manejan la mayoría de los casos de uso de propósito general. Estas aplican la lógica de transformaciones a los RDD existentes y generan un nuevo RDD de las operaciones comunes de agregación, filtros, etc. Ejemplos: map, filter, flatMap, groupByKey, sortByKey, combineByKey. 

   >-**Transformaciones matemáticas o estadísticas:** son funciones de transformación que manejan alguna funcionalidad estadística y que generalmente aplican alguna operación matemática o estadística en el RDD existente, generando un nuevo RDD. El muestreo es un gran ejemplo de esto y se usa a menudo en las aplicaciones de spark. Ejemplos: sampleByKey, randomSplit.

   >-**Transformaciones de conjunto o relacionales:** son funciones de que manejan transformaciones como uniones de conjuntos de datos y otras funciones algebraicas relacionales como grupo. Funcionan aplicando la lógica de transformación a los RD existentes y generando un nuevo RDD. Ejemplos: cogroup, join, subtractByKey, fullOuterJoin, leftOuterJoin, rightOuterJoin.
   
   >-**Transformaciones basadas en estructura de datos:** son funciones de transformación que operan en las estructuras de datos subyacentes del RDD, como las particiones por ejemplo. Estas funciones pueden trabajar directamente en particiones sin tocar directamente los elementos o los datos dentro del RDD. Por lo general, las mejoras de rendimiento se pueden realizar redistribuyendo las particiones de datos de acuerdo con el estado del plotter, el tamaño de los datos y los requisitos exactos del caso de uso. Ejemplos: partitionBy, repartition, zipwithIndex, coalesce.

### Función map

Map aplica la función de transformación que le proporcionamos a las particiones de entrada para generar particiones de salida en el RDD de salida. Cada partición del RDD da como resultado una nueva partición en un nuevo RDD que esencialmente aplica las transformaciones a todos los elementos del RDD.
Se le aplican funciones lamba.

In [47]:
#Creamos un RDD

rdd = sc.parallelize([1,2,3,4,5,6,7,8,9])

In [48]:
#Con map aplicaremos funciones lambda

rdd_resta = rdd.map(lambda x: x - 1)
rdd_resta.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [49]:
rdd_par = rdd.map(lambda x: x % 2 == 0)
rdd_par.collect()

[False, True, False, True, False, True, False, True, False]

In [50]:
rdd_texto = sc.parallelize(['jose', 'juaquin', 'juan', 'lucia', 'karla', 'katia'])

In [51]:
rdd_mayuscula = rdd_texto.map(lambda x: x.upper())
rdd_mayuscula.collect()

['JOSE', 'JUAQUIN', 'JUAN', 'LUCIA', 'KARLA', 'KATIA']

In [52]:
rdd_hola = rdd_texto.map(lambda x: 'Hola ' + x)
rdd_hola.collect()

['Hola jose',
 'Hola juaquin',
 'Hola juan',
 'Hola lucia',
 'Hola karla',
 'Hola katia']

### Función flatMap

Aplica la función de transformación a las particiones de entrada para generar particiones de salida en el RDD de salida, al igual que la función map. Sin embargo, flatMap también aplana o hace un floating de cualquier colección en los elementos del RDD de entrada.
Se le aplican funciones lamba.

In [53]:
#Aquí el resultado será un RDD con tuplas conteniendo x en la primera posición y x**2 en la segunda
rdd_cuadrado = rdd.map(lambda x: (x, x ** 2))
rdd_cuadrado.collect()

[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25), (6, 36), (7, 49), (8, 64), (9, 81)]

In [54]:
#Si aplicamos la lambda anterior con flatMap, nos devolverá el resultado sin tuplas, como elementos en una lista
rdd_cuadrado_flat = rdd.flatMap(lambda x: (x, x ** 2))
rdd_cuadrado_flat.collect()

[1, 1, 2, 4, 3, 9, 4, 16, 5, 25, 6, 36, 7, 49, 8, 64, 9, 81]

In [55]:
rdd_mayuscula = rdd_texto.flatMap(lambda x: (x, x.upper()))
rdd_mayuscula.collect()

['jose',
 'JOSE',
 'juaquin',
 'JUAQUIN',
 'juan',
 'JUAN',
 'lucia',
 'LUCIA',
 'karla',
 'KARLA',
 'katia',
 'KATIA']

### Función filter

Aplica la función de transformación a las particiones de entrada para generar particiones de salida filtradas en el RDD de salida.
Se le aplican funciones lamba.

In [56]:
#Filtramos para crear un nuevo RDD que contenga sólo los números pares
rdd_par = rdd.filter(lambda x: x % 2 == 0)
rdd_par.collect()

[2, 4, 6, 8]

In [57]:
#Filtramos para crear un nuevo RDD que contenga sólo los números impares
rdd_impar = rdd.filter(lambda x: x % 2 != 0)
rdd_impar.collect()

[1, 3, 5, 7, 9]

In [58]:
#Filtramos para crear un nuevo RDD que contenga sólo los nombres que empiecen con k
rdd_k = rdd_texto.filter(lambda x: x.startswith('k'))
rdd_k.collect()

['karla', 'katia']

In [59]:
#Filtramos para crear un nuevo RDD que contenga sólo los nombres que empiecen por j y que su segunda letra sea u
rdd_filtro = rdd_texto.filter(lambda x: x.startswith('j') and x.find('u') == 1)
rdd_filtro.collect()

['juaquin', 'juan']

### Función coalesce

Aplica una función de transformación a las particiones de entrada para combinar las particiones de entrada en menos particiones en el RDD de salida. Se crea un nuevo RD a partir del RD original, reduciendo a cantidad de particiones y combinándolas según sea necesario. Es una versión optimizada de repartición, donde el movimiento de los datos a través de las particiones es menor.

In [60]:
#Creamos un RDD con 10 particiones
rdd_part10 = sc.parallelize([1,2,3.4,5], 10)
rdd_part10.getNumPartitions()

10

In [61]:
# Un error habitual es olvidar que los RDD son inmutables, así que si intentas aplicar coalesce al RDD original, no ocurrirá nada
rdd_part10.coalesce(5)
rdd_part10.getNumPartitions()

10

In [62]:
#Para modificar el número de particiones, habrá que crear un nuevo RDD
rdd_part5 = rdd_part.coalesce(5)
rdd_part5.getNumPartitions()

5

### Función repartition

Aplica una función de transformación a las particiones de entrada para repartir la entrada en menos o más particiones de salida en el RDD de salida. Se crea un nuevo RDD a partir del RDD original, redistribuyendo las particiones, ya sea combinándolas o dividiéndola según sea necesario.

In [63]:
#Creamos un RDD con tres particiones
rdd_part3 = sc.parallelize([1,2,3,4,5], 3)
rdd_part3.getNumPartitions()

3

In [64]:
#Incrementamos en número de partiicones a 7
rdd_part7 = rdd_part3.repartition(7)
rdd_part7.getNumPartitions()

7

Un **punto importante** a tener en cuenta es que repartition y coalesce son operaciones muy costosas ya que mezclan los datos en muchas particiones, por lo tanto intente minimizar el uso de estos tanto como sea posible.

### Función reduceByKey

Se usa para fusionar los valores de cada clave usando una función asociativa de reducción. Es una transformación de tipo amplia o builder, ya que intercambia datos en múltiples particiones y opera por el par llave-valor. Se hace una reducción por la llave en la partición original y luego se realiza la reducción entre particiones, asignando el resultado a las particiones correspondientes.
Requiere una lambda que reciba dos valores, uno para el key y otro para el value.

In [66]:
#Creamos un RDD tipo llave-valor
rdd_keyValue = sc.parallelize(
    [('casa', 2),
     ('parque', 1),
     ('que', 5),
     ('casa', 1),
     ('escuela', 2),
     ('casa', 1),
     ('que', 1)]
)
rdd_keyValue.collect()

[('casa', 2),
 ('parque', 1),
 ('que', 5),
 ('casa', 1),
 ('escuela', 2),
 ('casa', 1),
 ('que', 1)]

In [67]:
#Sumamos los values de las keys repetidas
rdd_reducido = rdd_keyValue.reduceByKey(lambda x,y: x + y)
rdd_reducido.collect()

[('que', 6), ('escuela', 2), ('casa', 4), ('parque', 1)]